In [ ]:
%pylab inline
import time
import pandas as pd
import pysumma as ps
import xarray as xr
from matplotlib import cm
import itertools
import seaborn as sns
from pathlib import Path
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
from tqdm import tqdm_notebook as tqdm
sns.set_context('talk')

mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 6)
colors = ['black', 'lime', 'wheat', 'orange', 'peru', 'skyblue', 'dodgerblue', 'royalblue', 'violet', 'deeppink', 'crimson']
from cycler import cycler
cc = cycler(color=colors)

mpl.rcParams['axes.prop_cycle'] = cc
%run ../lib/summa_snow_layering.py

In [ ]:
dt_list = ['-2.0K', '+0.0K', '+2.0K', '+4.0K']
model_list = ['JRDN', 'CLM', '2L_thin', '2L_mid', '2L_thick', '3L_thin', '3L_mid', '3L_thick', '4L_thin', '4L_mid', '4L_thick']
site_list = ['Dana', 'Reynolds_Creek', 'Col-de-Port']
fname_mapper = {
    'Dana'           : './dana/output/template_output_DT_MODEL_timestep.nc',
    'Reynolds_Creek' : './reynolds/output/reynolds_DT_MODEL_timestep.nc',
    'Col-de-Port'    : './coldeport/output/template_output_DT_MODEL_timestep.nc',
}

fname_in_mapper = {
    'Dana': './dana/forcings/dana_forcing_DT.nc',
    'Reynolds_Creek': './reynolds/forcing/forcing_above_aspen_DT.nc',
    'Col-de-Port': './coldeport/forcings/col-de-port_1993-2011_DT.nc'
}

In [ ]:
dana_force_files = [fname_in_mapper['Dana'].replace('DT', dt) for dt in dt_list]
reynolds_force_files = [fname_in_mapper['Reynolds_Creek'].replace('DT', dt) for dt in dt_list]
coldeport_force_files = [fname_in_mapper['Col-de-Port'].replace('DT', dt) for dt in dt_list]

dana_files = [fname_mapper['Dana'].replace('DT', dt).replace('MODEL', model) for dt in dt_list for model in model_list]
reynolds_files = [fname_mapper['Reynolds_Creek'].replace('DT', dt).replace('MODEL', model) for dt in dt_list for model in model_list]
coldeport_files = [fname_mapper['Col-de-Port'].replace('DT', dt).replace('MODEL', model) for dt in dt_list for model in model_list]

dana_force_list = [xr.open_dataset(file).load() for file in tqdm(dana_force_files)]
reynolds_force_list = [xr.open_dataset(file).load() for file in tqdm(reynolds_force_files)]
coldeport_force_list = [xr.open_dataset(file).load() for file in tqdm(coldeport_force_files)]

dana_ds_list = [subset_and_aggregate_ds(xr.open_dataset(file).load()) for file in tqdm(dana_files)]
reynolds_ds_list = [subset_and_aggregate_ds(xr.open_dataset(file).load()) for file in tqdm(reynolds_files)]
coldeport_ds_list = [subset_and_aggregate_ds(xr.open_dataset(file).load()) for file in tqdm(coldeport_files)]

In [ ]:
new_coords = {'dt': dt_list, 'model': model_list}
dim_tuple = itertools.product(dt_list, model_list)
idx = pd.MultiIndex.from_tuples(dim_tuple, names=('dt', 'model'))

cdp_ds_concat = xr.concat(coldeport_ds_list, dim='new')
dana_ds_concat = xr.concat(dana_ds_list, dim='new')
reynolds_ds_concat = xr.concat(reynolds_ds_list, dim='new')

cdp_force_concat = xr.concat(coldeport_force_list, dim='new')
dana_force_concat = xr.concat(dana_force_list, dim='new')
reynolds_force_concat = xr.concat(reynolds_force_list, dim='new')

cdp_ds_concat['new'] = idx
cdp_merged = cdp_ds_concat.unstack('new').drop('hru')

dana_ds_concat['new'] = idx
reynolds_ds_concat['new'] = idx

dana_merged = dana_ds_concat.unstack('new').drop('hru')
reynolds_merged = reynolds_ds_concat.unstack('new').drop('hru')

In [ ]:
dana_force_ds = xr.concat(dana_force_list, dim='dt').isel(hru=0, drop=True)
dana_force_ds['time'] = dana_force_ds['time'].dt.round('H')

reynolds_force_ds = xr.concat(reynolds_force_list, dim='dt').isel(hru=0, drop=True)
reynolds_force_ds['time'] = reynolds_force_ds['time'].dt.round('H')

cdp_force_ds = xr.concat(coldeport_force_list, dim='dt').isel(hru=0, drop=True)
cdp_force_ds['time'] = cdp_force_ds['time'].dt.round('H')

dana_force_ds['dt'] = dt_list
reynolds_force_ds['dt'] = dt_list
cdp_force_ds['dt'] = dt_list

In [ ]:
dana_merged['time'] = dana_merged['time'].dt.round('H')
dana_force_ds = dana_force_ds.sel(time=dana_merged['time'])
dana_merged = xr.merge([dana_force_ds, dana_merged])

reynolds_merged['time'] = reynolds_merged['time'].dt.round('H')
reynolds_force_ds = reynolds_force_ds.sel(time=reynolds_merged['time'])
reynolds_merged = xr.merge([reynolds_force_ds, reynolds_merged])

cdp_merged['time'] = cdp_merged['time'].dt.round('H')
cdp_force_ds = cdp_force_ds.sel(time=cdp_merged['time'])
cdp_merged = xr.merge([cdp_force_ds, cdp_merged])

In [ ]:
cdp_merged.to_netcdf('../processed/coldeport_perturbations.nc')
dana_merged.to_netcdf('../processed/dana_perturbations.nc')
reynolds_merged.to_netcdf('../processed/reynolds_perturbations.nc')